# 온도 회귀

---

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_absolute_error

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
#temp = pd.read_csv('/content/drive/MyDrive/dacon_kogas/api/total_서울.csv',usecols=['시간','기온'])
temp_t = pd.read_csv('/content/drive/MyDrive/dacon_kogas/api/total_서울_test.csv',usecols=['시간','기온'])
temp = pd.read_csv('/content/drive/MyDrive/dacon_kogas/data/temp_1988_서울.csv')


In [28]:
temp.columns

Index(['일시', 'date', 'time', 'temp', 'year', 'month', 'day', 'weekday',
       'lunYear', 'lunMonth', 'lunDay'],
      dtype='object')

In [29]:
temp['date'].value_counts()

2005-06-05    24
1990-01-26    24
2001-02-09    24
2009-08-20    24
1998-09-26    24
              ..
2006-03-11    24
2004-12-02    24
2011-07-16    24
1990-05-08    24
1993-11-18    24
Name: date, Length: 11049, dtype: int64

In [30]:
test_y = temp_t['기온']

### 4.2. 학습, 테스트 데이터 분리

In [31]:
#temp.columns=['일시', 'date', 'time', 'temp', 'year', 'month', 'day', 'weekday',
#       'lunYear', 'lunMonth', 'lunDay']


In [32]:
df1 = temp.copy()

In [33]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265176 entries, 0 to 265175
Data columns (total 11 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   일시        265176 non-null  object 
 1   date      265176 non-null  object 
 2   time      265176 non-null  int64  
 3   temp      265176 non-null  float64
 4   year      265176 non-null  int64  
 5   month     265176 non-null  int64  
 6   day       265176 non-null  int64  
 7   weekday   265176 non-null  int64  
 8   lunYear   265176 non-null  int64  
 9   lunMonth  265176 non-null  int64  
 10  lunDay    265176 non-null  int64  
dtypes: float64(1), int64(8), object(2)
memory usage: 22.3+ MB


머신러닝의 성능을 평가 하기 위해서는 전체 데이터를 학습에 사용하지 않고 학습용 데이터와 테스트용 데이터를 나누어 사용합니다.

In [34]:
############ train/test 데이타 feature 지정 
from sklearn.model_selection import train_test_split
#df1['temp'] = df1['temp'].astype(int)

train_years = [ 
               #1988,1989, 1990,
              1991,1992,
               1993,1994,1995,1996,1997,1998,1999,2000,
               #2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
               
               #,2016,2017,2018
               ]
#val_years = [2018] 

train = temp[temp['year'].isin(train_years)]
#val = temp[temp['Year'].isin(val_years)]

features = ['lunMonth',	'lunDay','time','month'] 

train_x = train[features]
train_y = train['temp']

#val_x = val[features]
#val_y = val['temp']

print(train_x.shape, train_y.shape)
#print(val_x.shape, val_y.shape)

(87672, 4) (87672,)


In [37]:
# test_x 만들기
test_x = pd.read_csv('/content/drive/MyDrive/dacon_kogas/data/test_set_all.csv',usecols=['lunMonth','lunDay','time','Month'])
test_x.columns=['lunMonth','lunDay','time','month']

## 5. 머신러닝 모델 학습

In [38]:
learning_rate = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
#param_grid = dict(learning_rate=learning_rate)

model = XGBRegressor(learning_rate=0.17, ) # XGB 모델
model.fit(train_x, train_y)

[13:25:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(learning_rate=0.07)

In [39]:
prediction7 = model.predict(test_x)
mean_absolute_error(test_y, prediction7)   #0.05: 2.8660751285845483 #0.06: 2.836437865062444  #0.07: 2.8286048355726177 ### 0.08: 2.8518813311253433  ### 0.09: 2.838690427166444  #0.095:2.849312175037684 0.1 :2.858637726927797 # 0.11 2.8689084969874887
# 0.06~0.095 
# best: 0.07: 2.8286048355726177  3년제외: 2.8241033095231765 4년제외: 2.830483863326134 5년제외: 2.876 

### 2.8392807118456673  ###0.67: 2.837077147251478 # 2.853787141977637

10.01218869112708

In [14]:
prediction_xgb = pd.DataFrame({'temp':prediction7})
prediction_xgb

,temp
0,-1.501578
1,-1.501578
2,-1.674146
3,-2.002753
4,-2.394333
...,...
2155,11.872465
2156,10.866207
2157,10.016950
2158,9.398530


In [15]:
prediction_date = pd.read_csv('/content/drive/MyDrive/dacon_kogas/data/test_set_all.csv',usecols=['일시'])

In [16]:
test_temp = pd.concat([prediction_date,prediction_xgb], axis=1)

In [17]:
temp_10 = pd.read_csv('/content/drive/MyDrive/dacon_kogas/weather_10_float.csv')
temp_10.columns=['일시','temp']

In [18]:
temp_all = pd.concat([temp_10[:240], test_temp[240:]],axis=0)

In [19]:
mean_absolute_error(test_y, temp_all['temp'].values) #2.7552534182918538   ### 0.09 : 2.728431309922119  ##0.07: 2.7139491299233196  ##2018년제외: 2.7094178427366438, 2017,18 제외: 2.7122424370654445
## 3년제외: 2.701484768191667  ###2.714237817304799

2.6389178596737457

In [20]:
temp_all.to_csv('/content/drive/MyDrive/dacon_kogas/temp_XGB_30yr__3yrex_2.701.csv',index=False)  ### 2.701

In [21]:
temp_all.to_csv('/content/drive/MyDrive/dacon_kogas/temp_XGB_30yr__3yrex_2.714.csv',index=False)  ### 2.714 model = XGBRegressor(learning_rate=0.07) #,2016,2017,2018